In [15]:
import cv2
import numpy as np
from tqdm import tqdm

In [16]:
IN_DIR = "E:\\M2\\original\\pool\\20221028_wada\\"
OUT_DIR = "E:\\M2\\result\\20221028\\"
sub_dir_list = [
    "dual_2_40_1_C001H001S0001",
    "dual_2_40_2_C001H001S0001",
    "dual_2_40_3_C001H001S0001",
    "dual_2_40_4_C001H001S0001",
    "dual_2_40_5_C001H001S0001",
    "dual_2_40_6_C001H001S0001"
]
sub_dir_list2 = [
    "dual_2\\1\\",
    "dual_2\\2\\",
    "dual_2\\3\\",
    "dual_2\\4\\",
    "dual_2\\5\\",
]

In [17]:
p_original = np.float32([[404, 434], [728, 440], [360, 515], [772, 518], ])
p_trans = np.float32([[800, 1100], [1320, 1100], [800, 1400], [1320, 1400]])
M = cv2.getPerspectiveTransform(p_original, p_trans)

In [18]:
c = (557, 770)
rot_mat = cv2.getRotationMatrix2D(c, -7, 1.0)

In [19]:
mask = cv2.imread("m.bmp", 0)
mask2 = cv2.imread("MASK_dual_2_40_1_C001H001S0001.bmp", 0)

In [20]:
def preprocessing_2(img, mode):
    if mode == 0 or mode == 3:
        img = cv2.bitwise_and(img, img, mask=mask)
        img = cv2.bitwise_and(img, img, mask=mask2)
        img = cv2.warpPerspective(img, M, (2000, 2000))
        img = cv2.warpAffine(img, rot_mat, img.shape[1::-1])
        _, img = cv2.threshold(img[500:1800, :], 150, 255, cv2.THRESH_BINARY)
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    else:
        img = cv2.bitwise_and(img, img, mask=mask)
        img = cv2.warpPerspective(img, M, (2000, 2000))
        img = cv2.warpAffine(img, rot_mat, img.shape[1::-1])
        _, img = cv2.threshold(img[500:1800, :], 90, 255, cv2.THRESH_BINARY)
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img)

    return img, centroids

In [21]:
for d in range(4, 5):
    for i in tqdm(range(1, 10900)):
        dst, res = preprocessing_2(img=cv2.imread(IN_DIR + sub_dir_list[d] + "\\" + sub_dir_list[d] + f"{i:06}" + ".bmp", 0), mode=d)
        cv2.imwrite(OUT_DIR + "img\\" + sub_dir_list2[d] + f"{i}.png", dst)
        np.savetxt(OUT_DIR + "pp\\" + sub_dir_list2[d] + f"pp_{i}.csv", res[1:], delimiter=',')

100%|██████████| 10899/10899 [31:30<00:00,  5.77it/s] 
